<a href="https://colab.research.google.com/github/hellomomiji/info7374-llm/blob/main/Assignment_3_yang_jiang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment 3

100 points

Implement the “Self Alignment with Instruction Backtranslation” paper. When fine tuning the model, use LoRA. You will not be able to do full finetuning because there is not enough memory.

Link to paper: https://arxiv.org/pdf/2308.06259.pdf
Colab’s GPU usage is limited. Try to first prototype and get things working on the CPU first before training on the GPU with the full dataset. If you are not able to connect to a GPU on colab, you can try to create a PyTorch Lightning Studio or a Kaggle notebook.

In particular:

1. Finetune the base language model (llama2 7B) with (output, instruction) pairs {(yi, xi)} from the seed data to obtain a backward model Myx := p(x|y). In other words, finetune a model that uses the output to predict the instruction. Use the openassistant-guanaco training set dataset. (25 points)

- Push the backwards model to HF and paste url here
- https://huggingface.co/hellomomiji/llama-2-7b-chat-guanaco

2. Self-Augmentation -- Randomly sample a subset of size 150 and generate instructions from the LIMA dataset’s completions and filtering out any mutli-turn examples. Print out 5 examples of generated instructions. (25 points)

- (generated instructions from backwards model, response is from LIMA) pairs

- Single turn:
  - Single turn: (What is the capital of France?, Paris)
  - Multi turn: (What is the meaning of life, 42, Why is it 42?, That’s universe, ...)

3. Self curation (selecting high quality examples) using few shot prompting in addition to the prompt in Table 1 of the paper. Print out 5 examples of high quality examples and 5 examples of low quality examples. (25 points)

- Push the dataset to HF hub and paste the url here
https://huggingface.co/datasets/hellomomiji/filtered_dataset

- Goal is to filter out bad samples
- Method: using an LLM to rate the example
  - LLM (meta/llama-7b-chat-hf): LLM(“Evaluate the quality of the instruction/response pair” + example.” Rate it from 1-5)

4. Finetune base model on dataset generated by step 3. Print out 5 example responses. (25 points)
- Push the instruction fine tuned model to HF hub and paste the url here
https://huggingface.co/hellomomiji/llama-2-7b-chat-guanaco-filtered

Please include a link to your colab notebook here:
https://colab.research.google.com/drive/1vMVOth4kH1BgpW2Myqddj3g11jIxBdbd?usp=sharing




In [ ]:
!pip install -q -U bitsandbytes wandb datasets sentence_transformers faiss-gpu
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U git+https://github.com/huggingface/trl.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from torch import nn as nn
from torch.nn import functional as F
from torch import optim

Finetune the base language model (llama2 7B) with (output, instruction) pairs {(yi, xi)} from the seed data to obtain a backward model Myx := p(x|y). In other words, finetune a model that uses the output to predict the instruction. Use the openassistant-guanaco training set dataset. (25 points)

In [ ]:
# Load model
model_name = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


# Load dataset
raw_dataset = load_dataset('timdettmers/openassistant-guanaco')
raw_dataset = raw_dataset['train'].train_test_split(test_size=0.1)
raw_dataset = raw_dataset.filter(lambda x: len(tokenizer.tokenize(x['text'])) < 256)

print(raw_dataset)

# prompt format for tuning backward the model
prompt_backward = """<s>[INST] This is a response to an instruction. Write an instruction that best suits the given response.
### Response:
{}
### Instruction:
{}[/INST]
"""

prompt_forward = """<s>[INST] This is an instruction for a given response. Write a response that best suits the given instruction.
### Instruction:
{}
### Response:
{}[/INST]
"""

EOS_TOKEN = tokenizer.eos_token


Repo card metadata block was not found. Setting CardData to empty.


Filter:   0%|          | 0/8861 [00:00<?, ? examples/s]

Filter:   0%|          | 0/985 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3033
    })
    test: Dataset({
        features: ['text'],
        num_rows: 334
    })
})


In [ ]:
print(f"Train: {raw_dataset['train'][0]['text']}")
print(f"Test: {raw_dataset['test'][0]['text']}")

Train: ### Human: What is the airspeed velocity of an unladen swallow?### Assistant: The airspeed velocity of an European unladen swallow has been approximated to be about 24 miles per hour or 11 meters per second. This was achieved by performing mathematical calculations based on body mass and wing length of the bird. A more accurate answer requires further experimentation.### Human: I would appreciate it if you could explain in detail what additional experiments are needed.
Test: ### Human: Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo?### Assistant: Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como 

In [ ]:
splits = ['train', 'test']
texts = []
texts_forward = []

for split in splits:
    for sample in raw_dataset[split]:
      text = sample['text']
      parts = text.split("### ")
      parts = parts[1:]

      if not (parts[0].startswith('Human: ') or parts[1].startswith('Assistant: ')):
          continue
      instruction = parts[0].replace("Human: ", "")
      response = parts[1].replace("Assistant: ", "")
      texts.append(prompt_backward.format(response, instruction) + EOS_TOKEN)
      texts_forward.append(prompt_forward.format(instruction, response) + EOS_TOKEN)

dataset = Dataset.from_dict({"text": texts})
dataset_forward = Dataset.from_dict({"text": texts_forward})


In [ ]:
dataset["text"][0]

'<s>[INST] This is a response to an instruction. Write an instruction that best suits the given response.\n### Response:\nThe airspeed velocity of an European unladen swallow has been approximated to be about 24 miles per hour or 11 meters per second. This was achieved by performing mathematical calculations based on body mass and wing length of the bird. A more accurate answer requires further experimentation.\n### Instruction:\nWhat is the airspeed velocity of an unladen swallow?[/INST]\n</s>'

In [ ]:
dataset_forward["text"][0]

'<s>[INST] This is an instruction for a given response. Write a response that best suits the given instruction.\n### Instruction:\nWhat is the airspeed velocity of an unladen swallow?\n### Response:\nThe airspeed velocity of an European unladen swallow has been approximated to be about 24 miles per hour or 11 meters per second. This was achieved by performing mathematical calculations based on body mass and wing length of the bird. A more accurate answer requires further experimentation.[/INST]\n</s>'

In [ ]:
max_length = 256
def generate_and_tokenize_prompt(prompt):
    result = tokenizer(
        prompt['text'],
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(generate_and_tokenize_prompt)
tokenized_dataset_forward = dataset_forward.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/3367 [00:00<?, ? examples/s]

Map:   0%|          | 0/3367 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset["text"][0]

'<s>[INST] This is a response to an instruction. Write an instruction that best suits the given response.\n### Response:\nThe airspeed velocity of an European unladen swallow has been approximated to be about 24 miles per hour or 11 meters per second. This was achieved by performing mathematical calculations based on body mass and wing length of the bird. A more accurate answer requires further experimentation.\n### Instruction:\nWhat is the airspeed velocity of an unladen swallow?[/INST]\n</s>'

In [ ]:
tokenized_dataset_forward["text"][0]

'<s>[INST] This is an instruction for a given response. Write a response that best suits the given instruction.\n### Instruction:\nWhat is the airspeed velocity of an unladen swallow?\n### Response:\nThe airspeed velocity of an European unladen swallow has been approximated to be about 24 miles per hour or 11 meters per second. This was achieved by performing mathematical calculations based on body mass and wing length of the bird. A more accurate answer requires further experimentation.[/INST]\n</s>'

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "fc1",
        "fc2",
        "dense",
        "lm_head"
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    lr_scheduler_type='cosine',
    max_steps=50,
    learning_rate=2e-5,
    optim="paged_adamw_8bit",
    logging_steps=5,
)


trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

Train Backward model

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
5,3.702300
10,2.642600
15,2.258700
20,2.693400
25,2.724100
30,3.117800
35,2.411500
40,1.990800
45,2.882600
50,2.479100


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=2.6902856826782227, metrics={'train_runtime': 259.3741, 'train_samples_per_second': 0.193, 'train_steps_per_second': 0.193, 'total_flos': 509465434521600.0, 'train_loss': 2.6902856826782227, 'epoch': 0.01485001485001485})

Train Forward model

In [ ]:
trainer_forward = Trainer(
    model=model,
    train_dataset=tokenized_dataset_forward,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer_forward.train()

Step,Training Loss
5,2.977400
10,2.117400
15,1.872400
20,2.155600
25,2.235200
30,2.525000
35,2.033000
40,1.613600
45,2.503000
50,2.083800


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=2.211632671356201, metrics={'train_runtime': 182.6804, 'train_samples_per_second': 0.274, 'train_steps_per_second': 0.274, 'total_flos': 509465434521600.0, 'train_loss': 2.211632671356201, 'epoch': 0.01485001485001485})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
new_model_name = "llama-2-7b-chat-guanaco"
new_model_name_forward= "llama-2-7b-chat-guanaco_forward"

trainer.tokenizer = tokenizer
trainer.model.save_pretrained(new_model_name)
trainer.tokenizer.save_pretrained(new_model_name)
trainer.model.push_to_hub(new_model_name)

trainer_forward.tokenizer = tokenizer
trainer_forward.model.save_pretrained(new_model_name_forward)
trainer_forward.tokenizer.save_pretrained(new_model_name_forward)


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/367M [00:00<?, ?B/s]

('llama-2-7b-chat-guanaco_forward/tokenizer_config.json',
 'llama-2-7b-chat-guanaco_forward/special_tokens_map.json',
 'llama-2-7b-chat-guanaco_forward/tokenizer.model',
 'llama-2-7b-chat-guanaco_forward/added_tokens.json',
 'llama-2-7b-chat-guanaco_forward/tokenizer.json')

Self-Augmentation -- Randomly sample a subset of size 150 and generate instructions from the LIMA dataset’s completions and filtering out any mutli-turn examples. Print out 5 examples of generated instructions. (25 points)

In [ ]:
dataset_lima=load_dataset('GAIR/lima')

In [ ]:
dataset_lima = dataset_lima.filter(lambda x: len((x['conversations'])) < 3)
print(dataset_lima)
print(dataset_lima['train'][0])

DatasetDict({
    train: Dataset({
        features: ['conversations', 'source'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['conversations', 'source'],
        num_rows: 300
    })
})
{'conversations': ['Can brain cells move? By movement I mean long distance migration (preferably within the brain only).', 'The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells

In [ ]:
instructions = []
max_conv = 150
i = 0
for conv in dataset_lima["train"]:
    if i > max_conv:
        break
    question, ans = conv["conversations"][0], conv["conversations"][1]

    # Prepare input prompt for the backward model

    prompt = f"""<s>[INST] Generate an instruction that best suits the given response. Your instruction should be short and concise.
    ### Response: {ans}
    ### Generated Instruction: [/INST]"""
    input_ids = trainer.tokenizer.encode(prompt, return_tensors="pt")

    # Generate instruction using the backward model
    generated_ids = trainer.model.generate(input_ids, max_new_tokens=512)
    generated_instruction = trainer.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    instructions.append({"question": question, "ans": ans, "generated_ques": generated_instruction})

    if i % 10 == 0:
        print(f"Completed {i} predictions")

    i += 1


examples = []
for instruction in instructions:
    examples.append({'instruction': instruction['generated_ques'], 'ans': instruction['ans']})


Completed 0 predictions
Completed 10 predictions
Completed 20 predictions
Completed 30 predictions
Completed 40 predictions
Completed 50 predictions
Completed 60 predictions
Completed 70 predictions
Completed 80 predictions
Completed 90 predictions
Completed 100 predictions
Completed 110 predictions
Completed 120 predictions
Completed 130 predictions
Completed 140 predictions
Completed 150 predictions


In [ ]:
# print(examples[0]['instruction'])
# print(examples[0]['ans'])

cleaned_examples = []
for example in examples:
  ans = example['ans']
  instruction = example['instruction'].split("### Generated Instruction:")[-1].replace("\n", "")
  instruction = instruction.split("[/INST]")[-1]
  cleaned_examples.append({'instruction': instruction, 'ans': ans})


In [ ]:
print(len(cleaned_examples))
for example in cleaned_examples[:5]:
  print(f"Instruction:\n {example['instruction']}")
  print(f"Answer:\n {example['ans']} \n")


151
Instruction:
   Instruction:Migrating brain cells are a common occurrence in both embryonic and adult brains. In the adult brain, glial cells, stem cells, and non-differentiated neurons have been shown to migrate. For example, oligodendrocytes migrate to form the insulating myelin sheath, while neuronal stem cells migrate in response to injury. Post-mitotic neurons have also been observed to migrate in the adult brain, including in fish, mammals, and non-human primates. During embryonic development, post-mitotic neurons destined for peripheral functions migrate over long distances from the neural crest to their target locations.
Answer:
 The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adu

Self curation (selecting high quality examples) using few shot prompting in addition to the prompt in Table 1 of the paper. Print out 5 examples of high quality examples and 5 examples of low quality examples. (25 points)

In [ ]:
self_curation_prompt = """Below is an instruction from an user and a candidate answer.
Evaluate whether or not the answer is a good example of how AI Assistant should respond to the user’s instruction.
Please assign a score using the following 5-point scale:
1:  It means the answer is incomplete, vague, off-topic, controversial, or not exactly what the user asked for.  For example, some content seems missing, numbered list does not start from the beginning, the opening sentence repeats user’s question. Or the response is from another person’s perspective with their personal experience (e.g.  taken from blog posts), or looks like an answer from a forum.  Or it contains promotional text, navigation text, or other irrelevant information.
2:  It means the answer addresses most of the asks from the user.  It does not directly address the user’s question.  For example, it only provides a high-level methodology instead of the exact solution to user’s question.
3:  It means the answer is helpful but not written by an AI Assistant.  It addresses all the basic asks from the user.  It is complete and self contained with the drawback that the response is not written from an AI assistant’s perspective, but from other people’s perspective.  The content looks like an excerpt from a blog post, web page, or web search results.  For example, it contains personal experience or opinion, mentions comments section, or share on social media, etc.
4:  It means the answer is written from an AI assistant’s perspective with a clear focus of addressing the instruction.  It provide a complete, clear, and comprehensive response to user’s question or instruction without missing or irrelevant information.  It is well organized, self-contained, and written in a helpful tone.  It has minor room for improvement, e.g.  more concise and focused.
5:  It means it is a perfect answer from an AI Assistant.  It has a clear focus on being a helpful AI Assistant, where the response looks like intentionally written to address the user’s question or instruction without any irrelevant sentences.  The answer provides high quality content, demonstrating expert knowledge in the area, is very well written, logical, easy-to-follow, engaging and insightful.
Please first provide a brief reasoning you used to derive the rating score, and then write "Score:  <rating>" in the last line.

Instruction: {}
Answer: {}
"""


max_iterations = 100
i = 0
curated_set = []
for example in cleaned_examples:
    if i > max_iterations:
        break
    prompt = self_curation_prompt.format(example["instruction"], example["ans"])
    input_ids = trainer_forward.tokenizer.encode(prompt, return_tensors="pt")
    generated_ids = trainer_forward.model.generate(input_ids, max_new_tokens=512)
    score = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    curated_set.append({"generated_question": example["instruction"], "answer": example["ans"], "score":score})

    if i % 10 == 0:
        print(f"Completed {i} predictions")

    i+=1

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2113: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Completed 0 predictions
Completed 10 predictions
Completed 20 predictions
Completed 30 predictions
Completed 40 predictions
Completed 50 predictions
Completed 60 predictions
Completed 70 predictions
Completed 80 predictions
Completed 90 predictions
Completed 100 predictions


In [ ]:

print("Question ", curated_set[0]['generated_question'])
print("Ans", curated_set[0]['answer'])
print("Score", curated_set[0]['score'])

Question    Instruction:Migrating brain cells are a common occurrence in both embryonic and adult brains. In the adult brain, glial cells, stem cells, and non-differentiated neurons have been shown to migrate. For example, oligodendrocytes migrate to form the insulating myelin sheath, while neuronal stem cells migrate in response to injury. Post-mitotic neurons have also been observed to migrate in the adult brain, including in fish, mammals, and non-human primates. During embryonic development, post-mitotic neurons destined for peripheral functions migrate over long distances from the neural crest to their target locations.
Ans The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.
How

In [ ]:
new_scores = []
print(curated_set[0]["score"].rsplit("Score:", 1)[-1].strip(" <>")[:1])
for sample in curated_set:
    try:
        score = float(sample["score"].rsplit("Score:", 1)[-1].strip(" <>")[:1])
        new_scores.append({
        "generated_question": sample["generated_question"],
        "answer": sample["answer"],
        "score": score
    })
    except ValueError as e:
        score = 0
        print(f"Error processing score for {sample}: {e}")

5
Error processing score for {'generated_question': "  The best instruction for this scenario is:Use `subprocess.check_call()` or `subprocess.run()` to open the file.Both of these functions are more secure and flexible than `os.system()`, as they allow you to specify the command to be executed and handle errors more effectively.For Python 2.7 and newer, `subprocess.check_call(['open', filename])` is a good choice.For Python 3.5+, you can use `subprocess.run(['open', filename], check=True)` for slightly more versatility.Avoid using `os.system()` as it is deprecated and does not provide adequate error handling.", 'answer': '```open``` and ```start``` are command-interpreter things for Mac OS/X and Windows respectively, to do this.\nTo call them from Python, you can either use ```subprocess``` module or ```os.system()```. \nHere are considerations on which package to use:\n\n* You can call them via ```os.system```, which works, but...\nEscaping: ```os.system``` only works with filenames t

In [ ]:
threshold = 4
filtered_scores = [x for x in new_scores if x["score"] >= threshold]

In [ ]:
len(filtered_scores)

76

In [ ]:
print(filtered_scores[0]['generated_question']+"\n")
print(filtered_scores[0]['answer']+"\n")
print(filtered_scores[0]['score'])

  Instruction:Migrating brain cells are a common occurrence in both embryonic and adult brains. In the adult brain, glial cells, stem cells, and non-differentiated neurons have been shown to migrate. For example, oligodendrocytes migrate to form the insulating myelin sheath, while neuronal stem cells migrate in response to injury. Post-mitotic neurons have also been observed to migrate in the adult brain, including in fish, mammals, and non-human primates. During embryonic development, post-mitotic neurons destined for peripheral functions migrate over long distances from the neural crest to their target locations.

The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.
However, after s

In [ ]:
prompt_forward = """<s>[INST] This is an instruction for a given response. Write a response that best suits the given instruction.
### Instruction:
{}
### Response:
{}
[/INST]
"""

EOS_TOKEN = tokenizer.eos_token

filtered_dataset = []

for d in new_scores:
    filtered_dataset.append({"text":prompt_forward.format(d["generated_question"], d["answer"])})

print(filtered_dataset[0])

{'text': '<s>[INST] This is an instruction for a given response. Write a response that best suits the given instruction.\n### Instruction:\n  Instruction:Migrating brain cells are a common occurrence in both embryonic and adult brains. In the adult brain, glial cells, stem cells, and non-differentiated neurons have been shown to migrate. For example, oligodendrocytes migrate to form the insulating myelin sheath, while neuronal stem cells migrate in response to injury. Post-mitotic neurons have also been observed to migrate in the adult brain, including in fish, mammals, and non-human primates. During embryonic development, post-mitotic neurons destined for peripheral functions migrate over long distances from the neural crest to their target locations.\n### Response:\nThe question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-s

In [ ]:
high_quality_examples = []
for d in filtered_scores:
  if d["score"] >= 5:
    high_quality_examples.append({"text":prompt_forward.format(d["generated_question"], d["answer"])})
  if len(high_quality_examples) == 5:
    break

for i, example in enumerate(high_quality_examples):
  print(f"### Example {i} ###\n")
  print(example["text"])


### Example 0 ###

<s>[INST] This is an instruction for a given response. Write a response that best suits the given instruction.
### Instruction:
  Instruction:Migrating brain cells are a common occurrence in both embryonic and adult brains. In the adult brain, glial cells, stem cells, and non-differentiated neurons have been shown to migrate. For example, oligodendrocytes migrate to form the insulating myelin sheath, while neuronal stem cells migrate in response to injury. Post-mitotic neurons have also been observed to migrate in the adult brain, including in fish, mammals, and non-human primates. During embryonic development, post-mitotic neurons destined for peripheral functions migrate over long distances from the neural crest to their target locations.
### Response:
The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fel

In [ ]:
bottom_5_examples = []
for d in filtered_scores:
  if d["score"] <= 4:
    bottom_5_examples.append({"text":prompt_forward.format(d["generated_question"], d["answer"])})

for i, example in enumerate(bottom_5_examples[:5]):
  print(f"### Example {i} ###\n")
  print(example["text"])

### Example 0 ###

<s>[INST] This is an instruction for a given response. Write a response that best suits the given instruction.
### Instruction:
  `csvgrep` and `csvcut` are not part of the standard Python toolkit, so you may need to install them using `pip install csvkit`.
### Response:
Sure, please take a look at csvkit. It provides a set of tools that adhere to the UNIX philosophy (meaning they are small, simple, single-purposed and can be combined). 

Here is an example that extracts the ten most populated cities in Germany from the free Maxmind World Cities database and displays the result in a console-readable format:
```$ csvgrep -e iso-8859-1 -c 1 -m "de" worldcitiespop | csvgrep -c 5 -r "\d+"
  | csvsort -r -c 5 -l | csvcut -c 1,2,4,6 | head -n 11 | csvlook
-----------------------------------------------------
|  line_number | Country | AccentCity | Population  |
-----------------------------------------------------
|  1           | de      | Berlin     | 3398362     |
|  2 

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
filtered_dataset = Dataset.from_dict({"text": filtered_dataset})
dataset_name = "filtered_dataset"
filtered_dataset.push_to_hub(dataset_name)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/297 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/hellomomiji/filtered_dataset/commit/f4a39c5491f7e0fbce811a4fe9f5f9f22d5511de', commit_message='Upload dataset', commit_description='', oid='f4a39c5491f7e0fbce811a4fe9f5f9f22d5511de', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/hellomomiji/filtered_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='hellomomiji/filtered_dataset'), pr_revision=None, pr_num=None)

Finetune base model on dataset generated by step 3 (25 points)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("hellomomiji/filtered_dataset")

dataset = dataset["train"].train_test_split(test_size=0.1)

print(dataset)

splits = ['train', 'test']
texts = []

for split in splits:
    for sample in dataset[split]['text']:
      text = sample['text'] + EOS_TOKEN
      texts.append(text)

dataset = Dataset.from_dict({'text': texts})

print(dataset['text'][0])

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 84
    })
    test: Dataset({
        features: ['text'],
        num_rows: 10
    })
})
<s>[INST] This is an instruction for a given response. Write a response that best suits the given instruction.
### Instruction:
  Based on the response provided, the best instruction for the given scenario is:"Assimilate"This instruction is concise and accurately reflects the various reasons why aliens might choose to assimilate into a new culture, including anthropology, tourism, reality TV, method acting, challenge, hermitage, camouflage, crash landing, and uplift.
### Response:
Aliens are alien, without a common background their motives may be incomprehensible.
If we assume that these aliens are Hollywood aliens (ie humans with odd-looking rubber bits stuck on their faces) then there are all of the reasons that humans from one culture spend time in another culture and try and learn to fit in:

* Anthropology; studyi

In [ ]:
max_length = 256
def generate_and_tokenize_prompt(prompt):
    result = tokenizer(
        prompt['text'],
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(generate_and_tokenize_prompt)


Map:   0%|          | 0/94 [00:00<?, ? examples/s]

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

new_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
)

new_model.config.use_cache = False
new_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "fc1",
        "fc2",
        "dense",
        "lm_head"
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

new_model = get_peft_model(new_model, config)

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    lr_scheduler_type='cosine',
    max_steps=50,
    learning_rate=2e-5,
    optim="paged_adamw_8bit",
    logging_steps=5,
)


new_trainer = Trainer(
    model=new_model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [ ]:
new_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hellomomiji (hellomomiji-northeastern-university). Use `wandb login --relogin` to force relogin


Step,Training Loss
5,2.299200
10,2.210700
15,2.290200
20,2.666100
25,2.324000
30,2.090600
35,2.386200
40,2.395900
45,2.358800
50,2.306500


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=2.3328240394592283, metrics={'train_runtime': 189.4253, 'train_samples_per_second': 0.264, 'train_steps_per_second': 0.264, 'total_flos': 509465434521600.0, 'train_loss': 2.3328240394592283, 'epoch': 0.5319148936170213})

In [ ]:
new_model_name = "llama-2-7b-chat-guanaco-filtered"

new_trainer.tokenizer = tokenizer
new_trainer.model.save_pretrained(new_model_name)
new_trainer.tokenizer.save_pretrained(new_model_name)

new_trainer.model.push_to_hub(new_model_name)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/367M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hellomomiji/llama-2-7b-chat-guanaco-filtered/commit/ab67eba20a763f47f17960d6b89ef2d61b7e4ded', commit_message='Upload model', commit_description='', oid='ab67eba20a763f47f17960d6b89ef2d61b7e4ded', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hellomomiji/llama-2-7b-chat-guanaco-filtered', endpoint='https://huggingface.co', repo_type='model', repo_id='hellomomiji/llama-2-7b-chat-guanaco-filtered'), pr_revision=None, pr_num=None)

In [ ]:
# Load model directly
new_model = new_trainer.model
new_model.eval()

# Response examples
prompt = """<s>[INST] This is an instruction for a given response. Write a response that best suits the given instruction. Instruction: {} [/INST]"""
instrution = "What is the capital of France?"
instruction_alternative = "What is the meaning of life?"
input_ids = tokenizer.encode(prompt.format(instrution), return_tensors="pt")
input_ids_alternative = tokenizer.encode(prompt.format(instruction_alternative), return_tensors="pt")

generated_ids = new_model.generate(input_ids, max_new_tokens=512)
generated_ids_alternative = new_model.generate(input_ids_alternative, max_new_tokens=512)

print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))
print(tokenizer.decode(generated_ids_alternative[0], skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2113: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


[INST] This is an instruction for a given response. Write a response that best suits the given instruction. Instruction: What is the capital of France? [/INST]  Sure! The capital of France is Paris.
[INST] This is an instruction for a given response. Write a response that best suits the given instruction. Instruction: What is the meaning of life? [/INST]  The meaning of life is a question that has puzzled philosophers, theologians, scientists, and everyday people for centuries. There are many different perspectives on what the meaning of life is, and there is no one definitive answer. However, here are some possible approaches to understanding the meaning of life:

1. Religious or spiritual perspective: Many people believe that the meaning of life is to fulfill a divine or spiritual purpose. According to this view, life has a higher purpose that is connected to a deity or a higher power. The meaning of life is to fulfill this purpose, which may involve following certain rules or practi